In [1]:
import pandas as pd
import psycopg2
from isbnlib import meta, is_isbn13

# Database
from neonconnect import connection_string

In [2]:
def connect():
    # Connect to the Postgres database
    conn = psycopg2.connect(connection_string)

    # Create a cursor object
    cur = conn.cursor()
    return cur, conn

In [3]:

def is_exists(isbn):
    ''' Returns 'True' if book already exists in the DB '''
    cur, conn = connect()
    query = f"SELECT id FROM app_title \
        WHERE isbn = '{isbn}'"
    cur.execute(query)
    id = cur.fetchone()
    if id:
        return id[0]
    return False

In [4]:
data = pd.read_excel('data_isbn.xlsx', index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 9781444937183 to 9780241534076
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        90 non-null     object 
 1   Author       90 non-null     object 
 2   Category     90 non-null     object 
 3   Level        74 non-null     object 
 4   Print Date   78 non-null     float64
 5   Location     0 non-null      float64
 6   Quantity     90 non-null     int64  
 7   Description  64 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 6.3+ KB


In [7]:
arr = list(data['Category'].unique())
arr

['Children Fiction',
 'Bedtime Stories & Nursery Rhymes',
 'Children Non-Fiction',
 'Children Development']

In [23]:
cur, conn = connect()
category_list = list(data['Category'].unique())
for i in category_list:
    handle = i.lower().replace(" ","-")
    cat_query = f"INSERT INTO storage_category (category, handle) VALUES ('{i}', '{handle}')"
    cur.execute(cat_query)
    conn.commit()



In [26]:
list(data['Level'].unique())

[nan,
 'Age 5+',
 'Age 3+',
 'Age 0-5',
 'Ages 0-2',
 'Ages 4-8, Ages 9-12',
 'Ages 7+',
 'Pre-school',
 'Ages 3+',
 'Ages 3-5',
 'Ages 6-8',
 'Ages 0-5',
 'Level 1',
 'Level 2',
 'Ages 4+',
 'Level 3',
 'Ages 5+',
 'Ages 9-11',
 'Ages 4-11',
 'Ages 2-5']

In [33]:
cur, conn = connect()
level_list = list(data['Level'].unique())
for i in level_list:
    handle = str(i).lower().replace(" ","-")
    level_query = f"INSERT INTO storage_level (level, handle) VALUES ('{i}', '{handle}')"
    cur.execute(level_query)
    conn.commit()

In [34]:
cur.close()
conn.close()

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 9781444937183 to 9780241534076
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        90 non-null     object 
 1   Author       90 non-null     object 
 2   Category     90 non-null     object 
 3   Level        74 non-null     object 
 4   Print Date   78 non-null     float64
 5   Location     0 non-null      float64
 6   Quantity     90 non-null     int64  
 7   Description  64 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 6.3+ KB


In [44]:
data['Print Date'] = data["Print Date"].fillna(0)

In [45]:
data['Print Date'] = data['Print Date'].astype(int) 

In [48]:
data['Print Date'].nunique()

16

In [51]:
data['Location'].unique()

array([nan])

In [54]:
data['Location'] = data['Location'].fillna('-')
data['Location'].unique()

array(['-'], dtype=object)

In [96]:
data1 = data.copy(deep=True)

In [97]:
data1 = data1.reset_index(level=0)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         90 non-null     int64 
 1   Title        90 non-null     object
 2   Author       90 non-null     object
 3   Category     90 non-null     object
 4   Level        74 non-null     object
 5   Print Date   90 non-null     int64 
 6   Location     90 non-null     object
 7   Quantity     90 non-null     int64 
 8   Description  64 non-null     object
dtypes: int64(3), object(6)
memory usage: 6.5+ KB


In [98]:
data1['ISBN'] = data1['ISBN'].astype(str)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         90 non-null     object
 1   Title        90 non-null     object
 2   Author       90 non-null     object
 3   Category     90 non-null     object
 4   Level        74 non-null     object
 5   Print Date   90 non-null     int64 
 6   Location     90 non-null     object
 7   Quantity     90 non-null     int64 
 8   Description  64 non-null     object
dtypes: int64(2), object(7)
memory usage: 6.5+ KB


In [132]:
cur, conn = connect()
query = "SELECT id, category FROM storage_category"
cur.execute(query)
categories = dict(cur.fetchall())
categories

{1: 'Children book',
 3: 'Adult book',
 4: 'Children Fiction',
 5: 'Bedtime Stories & Nursery Rhymes',
 6: 'Children Non-Fiction',
 7: 'Children Development'}

In [138]:

conn.close()

In [137]:
cur, conn = connect()
query = "SELECT id, level FROM storage_level"
cur.execute(query)
levels = dict(cur.fetchall())
levels

{1: 'no level',
 2: 'nan',
 3: 'Age 5+',
 4: 'Age 3+',
 5: 'Age 0-5',
 6: 'Ages 0-2',
 7: 'Ages 4-8, Ages 9-12',
 8: 'Ages 7+',
 9: 'Pre-school',
 10: 'Ages 3+',
 11: 'Ages 3-5',
 12: 'Ages 6-8',
 13: 'Ages 0-5',
 14: 'Level 1',
 15: 'Level 2',
 16: 'Ages 4+',
 17: 'Level 3',
 18: 'Ages 5+',
 19: 'Ages 9-11',
 20: 'Ages 4-11',
 21: 'Ages 2-5'}

In [141]:
levels_inv = {value: key for key,value in levels.items()}
levels_inv

{'no level': 1,
 'nan': 2,
 'Age 5+': 3,
 'Age 3+': 4,
 'Age 0-5': 5,
 'Ages 0-2': 6,
 'Ages 4-8, Ages 9-12': 7,
 'Ages 7+': 8,
 'Pre-school': 9,
 'Ages 3+': 10,
 'Ages 3-5': 11,
 'Ages 6-8': 12,
 'Ages 0-5': 13,
 'Level 1': 14,
 'Level 2': 15,
 'Ages 4+': 16,
 'Level 3': 17,
 'Ages 5+': 18,
 'Ages 9-11': 19,
 'Ages 4-11': 20,
 'Ages 2-5': 21}

In [140]:
categories_inv = {value: key for key,value in categories.items()}
categories_inv

{'Children book': 1,
 'Adult book': 3,
 'Children Fiction': 4,
 'Bedtime Stories & Nursery Rhymes': 5,
 'Children Non-Fiction': 6,
 'Children Development': 7}

In [167]:
def clean_title(string):
    result = string.replace("'", "-")
    return result


In [171]:
for index, row in data1.iterrows():
    print(index, row)

0 ISBN                                               9781444937183
Title                        Kipper the Dog Collection 10 Books 
Author                                               Mick Inkpen
Category                                        Children Fiction
Level                                                        NaN
Print Date                                                  2016
Location                                                       -
Quantity                                                      15
Description    Coming in a handy Plastick bag, this is a wond...
Name: 0, dtype: object
1 ISBN                                               9781839130670
Title          Elmer 12 Classic Picture Books Collection Set ...
Author                                               David McKee
Category                                        Children Fiction
Level                                                     Age 5+
Print Date                                                     

In [200]:
for row in data1.itertuples():
    index, isbn, title, author, category, level, year, location, quantity, description = arr
    print(description)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [202]:

count = 0
for row in data1.itertuples():
    cur, conn = connect()
    arr = list(row)
    print("arr is", arr)
    index, isbn, title, author, category, level, year, location, quantity, description = arr
    title = clean_title(title)
    try:
        category_id = categories_inv.get(category, 1)
        level_id = levels_inv.get(level, 1)
        # description = description.replace("'","")
        query = f'''INSERT INTO storage_title (isbn, title, author, category_id_id, level_id_id, year, description, user_id, created, in_stock, is_active) VALUES ({isbn}, '{title}', '{author}', {category_id}, {level_id}, {year}, '{description}', 1, CURRENT_TIMESTAMP, TRUE, TRUE);'''
        cur.execute(query)
        conn.commit()
        count += 1
        print("QUERY IS ", query)
    except Exception as err:
        print(err)
        print("QUERY IS ", query)
    cur.close()
    query = ''


conn.close()

print('COUNT = ', count)

arr is [0, '9781444937183', 'Kipper the Dog Collection 10 Books ', 'Mick Inkpen', 'Children Fiction', nan, 2016, '-', 15, "Coming in a handy Plastick bag, this is a wonderful collection of 10 of Mick Inkpen's classic Kipper stories. A favourite with toddlers for over 25 years, the lovable dog Kipper celebrates his birthday; goes camping; chooses a new pet; and plays in the snow throughout these brilliant books. This collection is a barking brilliant gift that is sure to be cherished for a long time to come"]
syntax error at or near "s"
LINE 1: ...is is a wonderful collection of 10 of Mick Inkpen's classic ...
                                                             ^

QUERY IS  INSERT INTO storage_title (isbn, title, author, category_id_id, level_id_id, year, description, user_id, created, in_stock, is_active) VALUES (9781444937183, 'Kipper the Dog Collection 10 Books ', 'Mick Inkpen', 4, 1, 2016, 'Coming in a handy Plastick bag, this is a wonderful collection of 10 of Mick Inkpen'

In [158]:
cur, conn = connect()
query = "SELECT COUNT(id) FROM storage_title"
cur.execute(query)
titles = cur.fetchall()
print(titles)
cur.close()
conn.close()

[(7,)]


In [159]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         90 non-null     object
 1   Title        90 non-null     object
 2   Author       90 non-null     object
 3   Category     90 non-null     object
 4   Level        74 non-null     object
 5   Print Date   90 non-null     int64 
 6   Location     90 non-null     object
 7   Quantity     90 non-null     int64 
 8   Description  64 non-null     object
dtypes: int64(2), object(7)
memory usage: 6.5+ KB
